In [57]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        # =========== OLD APPROACH FOR FIXED NUM OF DIFFERENT FILTERS =========== #
        # self.conv_0 = nn.Conv2d(in_channels = 1, 
        #                         out_channels = n_filters, 
        #                         kernel_size = (filter_sizes[0], embedding_dim))
        
        # self.conv_1 = nn.Conv2d(in_channels = 1, 
        #                         out_channels = n_filters, 
        #                         kernel_size = (filter_sizes[1], embedding_dim))
        
        # self.conv_2 = nn.Conv2d(in_channels = 1, 
        #                         out_channels = n_filters, 
        #                         kernel_size = (filter_sizes[2], embedding_dim))
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        #text = [batch size, sent len]

        embedded = self.embedding(text)
        #embedded = [batch size, sent len, emb dim]

        embedded = embedded.unsqueeze(1)
        #embedded = [batch size, 1, sent len, emb dim]

        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]    
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]        
        #pooled_n = [batch size, n_filters]

        cat = self.dropout(torch.cat(pooled, dim = 1))
        #cat = [batch size, n_filters * len(filter_sizes)]
                
        return self.fc(cat)

In [58]:
class CNN1d(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.convs = nn.ModuleList([
                                    nn.Conv1d(in_channels = embedding_dim, 
                                              out_channels = n_filters, 
                                              kernel_size = fs)
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.permute(0, 2, 1)
        #embedded = [batch size, emb dim, sent len]
        
        conved = [F.relu(conv(embedded)) for conv in self.convs]
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))
        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

In [59]:
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets
import random
import numpy as np
import pandas as pd

SEED = 1234

random.seed(SEED)

# create fields in emotion dataset
TEXT = data.Field(tokenize = 'spacy', 
                  tokenizer_language = 'en_core_web_sm',
                  batch_first = True
                  )
# EMOTION = data.LabelField(dtype = torch.float)
LABEL = data.LabelField()

# tuples representative of tabular format
fields = [
          ('Formality', LABEL),
          ('Sentence', TEXT)
]

# LOAD EMOTION DATA
form_train, = data.TabularDataset.splits(
    path='/content',
    train='mturk_news_formality.csv',
    format='csv',
    fields=fields,
    skip_header=True
)

form_train, form_valid = form_train.split(random_state = random.seed(SEED), split_ratio=0.7)
form_train, form_test = form_train.split(random_state = random.seed(SEED), split_ratio=0.7)

In [60]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(form_train, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_
                 )

LABEL.build_vocab(form_train)

In [61]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

form_train_iterator, form_valid_iterator, form_test_iterator = data.BucketIterator.splits(
    (form_train, form_valid, form_test), 
    batch_size = BATCH_SIZE, 
    device = device,
    sort=False)

In [62]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 50
FILTER_SIZES = [3,4,5]
# OUTPUT_DIM = 1
OUTPUT_DIM = len(LABEL.vocab)
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

In [63]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 752,707 trainable parameters


In [64]:
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

# zero initial weights of <unk> and <pad>
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

# Training

In [65]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
# criterion = nn.BCEWithLogitsLoss()
criterion = nn.CrossEntropyLoss()
model = model.to(device)
criterion = criterion.to(device)

###  Helpers

In [66]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [67]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    top_pred = preds.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [68]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        predictions = model(batch.Sentence)#.squeeze(1)
        loss = criterion(predictions, batch.Formality)
        acc = categorical_accuracy(predictions, batch.Formality)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [69]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            # print(batch.__dict__)
            predictions = model(batch.Sentence)#.squeeze(1)
            loss = criterion(predictions, batch.Formality)
            acc = categorical_accuracy(predictions, batch.Formality)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [70]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [71]:
N_EPOCHS = 10
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train(model, form_train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, form_valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'CNN-formality.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 0s
	Train Loss: 1.573 | Train Acc: 33.85%
	 Val. Loss: 1.377 |  Val. Acc: 38.84%
Epoch: 02 | Epoch Time: 0m 0s
	Train Loss: 1.357 | Train Acc: 42.42%
	 Val. Loss: 1.375 |  Val. Acc: 36.05%
Epoch: 03 | Epoch Time: 0m 0s
	Train Loss: 1.264 | Train Acc: 48.45%
	 Val. Loss: 1.350 |  Val. Acc: 37.05%
Epoch: 04 | Epoch Time: 0m 0s
	Train Loss: 1.157 | Train Acc: 56.50%
	 Val. Loss: 1.348 |  Val. Acc: 36.94%
Epoch: 05 | Epoch Time: 0m 0s
	Train Loss: 1.106 | Train Acc: 57.77%
	 Val. Loss: 1.344 |  Val. Acc: 36.94%
Epoch: 06 | Epoch Time: 0m 0s
	Train Loss: 1.020 | Train Acc: 63.90%
	 Val. Loss: 1.296 |  Val. Acc: 38.84%
Epoch: 07 | Epoch Time: 0m 0s
	Train Loss: 0.950 | Train Acc: 66.19%
	 Val. Loss: 1.293 |  Val. Acc: 38.39%
Epoch: 08 | Epoch Time: 0m 0s
	Train Loss: 0.843 | Train Acc: 72.94%
	 Val. Loss: 1.291 |  Val. Acc: 38.06%
Epoch: 09 | Epoch Time: 0m 0s
	Train Loss: 0.786 | Train Acc: 75.69%
	 Val. Loss: 1.311 |  Val. Acc: 37.61%
Epoch: 10 | Epoch Time: 0m 0

In [72]:
# Validation
model.load_state_dict(torch.load('CNN-formality.pt'))

test_loss, test_acc = evaluate(model, form_test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 1.303 | Test Acc: 39.93%


## TESTING

In [75]:
import spacy
nlp = spacy.load('en_core_web_sm')

def predict_emotion(model, sentence, min_len = 5):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [76]:
predict_emotion(model, "This is really poor")

ValueError: ignored